# documentation
```
generative_models_via_sqsp
    |
    |_ utilities
        |
        |_ auxiliaries.py
        |
        |_ compiler.py
        |
        |_ grover_state_preparation.py
        |
        |_ kernels.py
        |
        |_ qcbm.py  
        |
        |_ quantum_gates.py    
        |
        |_ sampling.py
    |
    .
    .
    .  
  ```

In [1]:
import os
os.chdir('generative_models_via_sqsp')

In [2]:
import numpy as np
import scipy.sparse as sps
from config import *

## auxiliaries

### loss

`loss(samples, p_i_set)`

Compute the loss between predicted and target values.
    
**Args**
- `samples` (array-like): Predicted values.
- `p_i_set` (array-like): Target values.

**Returns**
- `float`: Computed loss value.

In [3]:
from utilities.auxiliaries import loss
from utilities.grover_state_preparation import *
from scipy.stats import beta
from qiskit import transpile
from qiskit_aer import Aer

# Global variables
alpha = 2
beta_ = 2
a = 0
b = 1

# Beta distribution target
p_i_set = beta.pdf(np.linspace(a, b, 2**m), alpha, beta_)
p_i_set /= p_i_set.sum()

# Example of samples from Grover state preparation circuit
thetas = get_grover_angles(p_i_set,m)
qc = state_expansion(m,thetas)
t_qc = transpile(qc, backend=Aer.get_backend("aer_simulator"))
job = Aer.get_backend("aer_simulator").run(t_qc, shots=1024)
counts = job.result().get_counts(qc)
total_counts = sum(counts.values())
samples = {key: value / total_counts for key, value in counts.items()}
num_states = 2**m-1
all_states = [format(i, f"0{m}b") for i in range(num_states)]
samples_full = {state: samples.get(state, 0) for state in all_states}
y = np.array([samples_full[state] for state in all_states])
y /= y.sum()

# Resulting loss
print(loss(y,p_i_set))

0.12349005086847445


### objective_function

`objective_function(thetas_to_optimize, idx_thetas_to_optimize, thetas, p_i_set, shots)`

Evaluates the objective function by updating parameterized angles, running a quantum circuit, and computing the loss.

**Args**
- `thetas_to_optimize` (list or array): Values to update in the thetas list.
- `idx_thetas_to_optimize` (List[int]): Indices in thetas to replace.
- `thetas` (list or array): Full parameter list for the circuit.
- `p_i_set` (array-like): Target probability distribution.
`shots` (int): Number of measurement shots for quantum execution.

**Returns**
- `float`: The computed objective value based on the loss function.

In [4]:
from utilities.auxiliaries import *
from scipy.optimize import minimize

# Initial angles and position of the angles to be optimized (specified)
idx_thetas_to_optimize = [1]
thetas_to_optimize = np.array(generate_parameters(len(idx_thetas_to_optimize), k=2))
thetas = get_grover_angles(p_i_set, m)

# Target beta distribution
p_i_set = beta.pdf(np.linspace(a, b, 2**m-1), alpha, beta_)
p_i_set /= p_i_set.sum()

# Minimization of the objective function with fixed parameters expept the specified ones
result = minimize(objective_function, thetas_to_optimize,
                      args=(idx_thetas_to_optimize, thetas, p_i_set, shots),
                      method=optimizer_type, options={"disp": False, "maxiter": max_iterations})

# Results
best_thetas = np.array(thetas.copy())
best_thetas[idx_thetas_to_optimize] = result.x
print("Optimal value:", result.fun)
print("Optimized angles:", result.x)
print("Optimal overall angles:", best_thetas)
print("Initial angles:", thetas)

Optimal value: 0.20438707014444155
Optimized angles: [8.24356109]
Optimal overall angles: [1.57079633 8.24356109 1.09601625 2.25438439 1.64930833 1.49228432
 0.88720827 3.14159265 1.73279428 1.63466954 1.58897915 1.55261351
 1.50692312 1.40879837 0.        ]
Initial angles: [1.5707963267948968, 2.0455764003604058, 1.0960162532293876, 2.25438438596734, 1.6493083342606727, 1.4922843193291206, 0.8872082676224539, 3.141592653589793, 1.732794281366022, 1.6346695366848385, 1.588979146878833, 1.55261350671096, 1.5069231169049548, 1.4087983722237711, 0.0]


### generate_parameters

`generate_parameters(n, k)`

Generate a list of random parameters.
    
**Args**
- `n` (int): Number of parameters to generate.
- `k` (float, optional): Scaling factor for the range (default is 2).

**Returns**
- `list`: A list of `n` randomly generated parameters.

In [5]:
from utilities.auxiliaries import generate_parameters

print(generate_parameters(2, 2))

[2.032595232978463, 5.34983578781294]


## compiler

### `compiler(ops, locs, n)`

Compiles operators into a specific Hilbert space.

**Args**
- `ops` (list or tuple): A list or tuple of operators to be applied to the system.
- `locs` (list or tuple): The qubit locations on which the operators will act.
- `n` (int): The total number of qubits in the quantum system.

**Returns**
- `scipy.sparse.csr_matrix`: The resulting sparse matrix after applying all operators.

In [6]:
from utilities.quantum_gates import sx, sz
from utilities.compiler import compiler

locs = [0, 1]
n = 2
result = compiler([sx, sz], locs, n)
print(result.todense())

[[ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]


### `_wrap_identity(data_list, num_bit_list)`

Helper function to apply identity operators to the Hilbert space.

**Args:**
- `data_list` (list): A list of operators to be applied to the quantum system.
- `num_bit_list` (list): A list containing the number of qubits on which each operator acts.

**Returns:**
- `scipy.sparse.csr_matrix`: The resulting sparse matrix after applying the operators.

**Raises:**
- `Exception`: If the length of `num_bit_list` is inconsistent with the number of operators.


In [7]:
from utilities.compiler import _wrap_identity

data_list = [sx]
num_bit_list = [1,1]
full_operator = _wrap_identity(data_list, num_bit_list)
print(full_operator.toarray())

[[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


### `initial_wf(num_bit)`


Generates the initial wave function |00...0> for a quantum system.

**Args:**
- `num_bit` (int): The number of qubits in the system.

**Returns:**
- `np.ndarray`: The initial wave function as a numpy array.

**Remarks:**
- The function returns the state vector |00...0>, which represents the quantum system being initialized to the ground state (all qubits in state |0>).

In [8]:
from utilities.compiler import initial_wf

print(initial_wf(1))

[1.+0.j 0.+0.j]


## grover_state_preparation

### `get_grover_angles(p_i_set, m)`


Calculates Grover's angles for a given set of probabilities.

**Args:**
- `p_i_set` (list or array-like): A list of probabilities corresponding to quantum states.
- `m` (int): The number of qubits used in the quantum circuit.

**Returns:**
- `list`: A list of rotation angles required for Grover's state preparation.

**Raises:**
- `ValueError`: If the computed number of angles does not match the required length for `m` qubits.

In [9]:
from utilities.grover_state_preparation import get_grover_angles

p_i_set = [0.25, 0.25, 0.25, 0.25]
m = 2
thetas = get_grover_angles(p_i_set, m)
print(thetas)

[1.5707963267948966, 1.5707963267948966, 1.5707963267948966]


### `state_expansion(m, thetas)`

Constructs a quantum circuit that applies rotations based on the calculated angles.

**Args:**
- `m` (int): The number of qubits in the circuit.
- `thetas` (list): A list of rotation angles computed for Grover's algorithm.

**Returns:**
- `QuantumCircuit`: A quantum circuit implementing the state preparation.

**Raises:**
- `ValueError`: If the number of angles does not match the required `2^m - 1`.

In [10]:
from utilities.grover_state_preparation import state_expansion

m = 2
thetas = [0.1, 0.2, 0.3]
qc = state_expansion(m, thetas)
print(qc)

     ┌─────────┐┌───┐           ┌───┐           ┌─┐   
q_0: ┤ Ry(0.1) ├┤ X ├─────■─────┤ X ├─────■─────┤M├───
     └─────────┘└───┘┌────┴────┐└───┘┌────┴────┐└╥┘┌─┐
q_1: ────────────────┤ Ry(0.2) ├─────┤ Ry(0.3) ├─╫─┤M├
                     └─────────┘     └─────────┘ ║ └╥┘
c: 2/════════════════════════════════════════════╩══╩═
                                                 1  0 


## kernels

### `mix_rbf_kernel(x, y, sigma_list)`

Computes a mixture of RBF kernels between two datasets.

**Args:**
- `x` (numpy.ndarray): Dataset `x`, shape `(n_samples_x, n_features)`.
- `y` (numpy.ndarray): Dataset `y`, shape `(n_samples_y, n_features)`.
- `sigma_list` (list or np.ndarray): List of sigma values for the RBF kernels.

**Returns:**
- `numpy.ndarray`: The kernel matrix computed between `x` and `y`.

**Raises:**
- `ValueError`: If any sigma values are non-positive.

In [11]:
from utilities.kernels import mix_rbf_kernel

# Example 1: 1D Case
x_1d = np.array([1, 2, 3])
y_1d = np.array([2, 3, 4])
sigma_list = [0.5, 1.0, 2.0]

kernel_matrix_1d = mix_rbf_kernel(x_1d, y_1d, sigma_list)
print("Kernel Matrix (1D case):\n", kernel_matrix_1d)

# Example 2: 2D Case
x_2d = np.array([[1, 2], [2, 3], [3, 4]])
y_2d = np.array([[2, 3], [3, 4], [4, 5]])
kernel_matrix_2d = mix_rbf_kernel(x_2d, y_2d, sigma_list)
print("\nKernel Matrix (2D case):\n", kernel_matrix_2d)

Kernel Matrix (1D case):
 [[1.75321088 0.52153036 0.11663163]
 [3.         1.75321088 0.52153036]
 [1.75321088 3.         1.75321088]]

Kernel Matrix (2D case):
 [[1.10974538 0.15398638 0.01123242]
 [3.         1.10974538 0.15398638]
 [1.10974538 3.         1.10974538]]


### `RBFMMD2`


Computes the squared Maximum Mean Discrepancy (MMD) using an RBF kernel.

**Methods**
- `__init__(sigma_list)`
Initializes the RBFMMD2 object with a list of sigma values.
- `compute(x, y)`
Computes the squared MMD between two datasets.

In [12]:
from utilities.kernels import mix_rbf_kernel, RBFMMD2

sigma_list = [0.5, 1.0, 2.0]
basis = np.linspace(0, 1, 10)
mmd = RBFMMD2(sigma_list, basis)
px = np.array([0.1, 0.15, 0.05, 0.2, 0.1, 0.05, 0.1, 0.1, 0.1, 0.05])
py = np.array([0.05, 0.1, 0.1, 0.15, 0.2, 0.05, 0.1, 0.05, 0.15, 0.05])
mmd_loss = mmd(px, py)
print("MMD^2 Loss:", mmd_loss)

MMD^2 Loss: 0.004228112853614


## qcbm

### `ArbitraryRotation`

Represents a quantum gate that applies arbitrary rotations to qubits in the quantum circuit. It can apply three rotations per qubit, represented by a list of rotation angles.

**Methods**
- `__init__(self, num_bit)`: Initializes the ArbitraryRotation instance with the specified number of qubits.
- `num_param`: Property that returns the number of parameters for the rotations (3 parameters per qubit).
- `toscr(self, theta_list)`: Transforms this block into a sequence of sparse CSR matrices based on the provided list of rotation angles.

In [13]:
from utilities.qcbm import ArbitraryRotation

num_bit = 2
rotation_gate = ArbitraryRotation(num_bit)
print(f"Number of parameters in ArbitraryRotation: {rotation_gate.num_param}")
theta_list = np.random.rand(6)
csr_matrices = rotation_gate.tocsr(theta_list)
for i, mat in enumerate(csr_matrices):
    print(f"CSR matrix {i} for rotation:")
    print(mat.toarray())

Number of parameters in ArbitraryRotation: 6
CSR matrix 0 for rotation:
[[ 0.87284363-0.23160505j  0.06384759-0.42476651j  0.        +0.j
   0.        +0.j        ]
 [-0.06384759-0.42476651j  0.87284363+0.23160505j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.87284363-0.23160505j
   0.06384759-0.42476651j]
 [ 0.        +0.j          0.        +0.j         -0.06384759-0.42476651j
   0.87284363+0.23160505j]]
CSR matrix 1 for rotation:
[[ 0.68585251-0.70740065j  0.        +0.j         -0.02803101-0.1685376j
   0.        +0.j        ]
 [ 0.        +0.j          0.68585251-0.70740065j  0.        +0.j
  -0.02803101-0.1685376j ]
 [ 0.02803101-0.1685376j   0.        +0.j          0.68585251+0.70740065j
   0.        +0.j        ]
 [ 0.        +0.j          0.02803101-0.1685376j   0.        +0.j
   0.68585251+0.70740065j]]


### `CNOTEntangler`

Applies a series of CNOT gates that entangle pairs of qubits. The entanglement is formed by applying a CNOT gate for each pair in the provided list.

**Methods**
- `__init__(self, num_bit, pairs)`: Initializes the CNOTEntangler instance with the number of qubits and the pairs for entangling.
  
- `num_param`: Property that returns the number of parameters (CNOT gates do not have parameters).

- `toscr(self, theta_list)`: Transforms this block into a sequence of sparse CSR matrices by applying CNOT gates to the specified qubit pairs.

In [14]:
from utilities.qcbm import CNOTEntangler

num_bit = 2
pairs = [(0, 1)]
cnot_entangler = CNOTEntangler(num_bit, pairs)
print(f"Number of parameters in CNOTEntangler: {cnot_entangler.num_param}")
theta_list = np.array([])
csr_matrices = cnot_entangler.tocsr(theta_list)
print("CNOTEntangler CSR matrix:")
print(csr_matrices[0].toarray())


Number of parameters in CNOTEntangler: 0
CNOTEntangler CSR matrix:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


### `BlockQueue`


Keeps track of the quantum circuit's evolution by managing blocks of operations and applying them to a quantum register while efficiently tracking the parameter changes.

**Methods**

- `__init__(self, *args)`: Initializes a BlockQueue instance with a sequence of quantum operations (blocks).
  
- `__call__(self, qureg, theta_list)`: Applies operations to the quantum register in place using the provided list of parameters.

- `num_bit`: Property that returns the number of qubits in the quantum circuit.

- `num_param`: Property that returns the total number of parameters across all blocks.

In [15]:
from utilities.qcbm import BlockQueue

num_bit = 2
pairs = [(0, 1)]
blocks = [ArbitraryRotation(num_bit), CNOTEntangler(num_bit, pairs)]
block_queue = BlockQueue(blocks)
print(f"Number of qubits: {block_queue.num_bit}")
print(f"Total number of parameters in BlockQueue: {block_queue.num_param}")
qureg = np.zeros(2**num_bit, dtype='complex128')
qureg[0] = 1.0
theta_list = np.random.rand(block_queue.num_param)
block_queue(qureg, theta_list)
print("Updated quantum register (wavefunction):")
print(qureg)

Number of qubits: 2
Total number of parameters in BlockQueue: 6
Updated quantum register (wavefunction):
[ 0.08478663-0.93885553j -0.00729165+0.00146758j -0.01789449-0.0111009j
 -0.20514207-0.26225842j]


### `QCBM`


The Quantum Circuit Born Machine framework that learns to approximate probability distributions using quantum circuits. The model uses rotation gates and CNOT entanglers, and the optimization is performed using the MMD loss function.

**Methods**
- `__init__(self, circuit, mmd, p_data, batch_size=None)`: Initializes the QCBM instance with the specified quantum circuit, MMD metric, target probability distribution, and batch size (optional).

- `depth`: Property that returns the depth of the circuit, defined by the number of entanglers.

- `pdf(self, theta_list)`: Gets the probability distribution function by applying the quantum circuit to the quantum state.

- `mmd_loss(self, theta_list)`: Computes the MMD loss for the given parameters.

- `gradient(self, theta_list)`: Computes the gradient of the MMD loss with respect to the parameters using numerical gradient computation.

In [16]:
from utilities.qcbm import ArbitraryRotation, CNOTEntangler, BlockQueue, QCBM

def dummy_mmd(prob1, prob2):
    return np.mean((prob1 - prob2) ** 2)

blocks = [ArbitraryRotation(num_bit), CNOTEntangler(num_bit, pairs)]
block_queue = BlockQueue(blocks)
p_data = np.random.rand(2**num_bit)
p_data /= p_data.sum()
qcbm = QCBM(circuit=block_queue, mmd=dummy_mmd, p_data=p_data)
theta_list = np.random.rand(block_queue.num_param)
prob_distribution = qcbm.pdf(theta_list)
print("Computed probability distribution:")
print(prob_distribution)

Computed probability distribution:
[0.76052607 0.00458274 0.21897479 0.01591641]


## quantum_gates


### `_ri(si, theta)`

Generates a single qubit rotation operator for a given angle.

**Args**
- `si` (scipy.sparse.csr_matrix): Pauli matrix (X, Y, or Z).
- `theta` (float): Rotation angle.

**Returns**
- `scipy.sparse.csr_matrix`: The single qubit rotation matrix.

In [17]:
from utilities.quantum_gates import _ri, sz

theta = np.pi / 4
rotation_sz = _ri(sz, theta)
print(rotation_sz.todense())

[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]


### `rot(t1, t2, t3)`

Generates a general rotation gate `rz(t3)rx(t2)rz(t1)`.

**Args**
- `t1` (float): Angle for the first rz rotation.
- `t2` (float): Angle for the rx rotation.
- `t3` (float): Angle for the second rz rotation.

**Returns**
- `scipy.sparse.csr_matrix`: The combined rotation gate.

In [18]:
from utilities.quantum_gates import rot

t1, t2, t3 = np.pi/4, np.pi/4, np.pi/4
full_rotation = rot(t1, t2, t3)
print(full_rotation.todense())

[[0.65328148-0.65328148j 0.        -0.38268343j]
 [0.        -0.38268343j 0.65328148+0.65328148j]]


###  `_rot_tocsr_update1(layer, old, theta_old, theta_new)`

rotation layer csr_matrices update method.

**Args**
- `layer` (ArbitraryRotation): rotation layer.
- `old` (csr_matrix): old matrices.
- `theta_old` (1darray): old parameters.
- `theta_new` (1darray): new parameters.

**Returns**
- `list of csr_matrix`: new rotation matrices after the theta changed.

### `CNOT(ibit, jbit, n)`

 Generates a CNOT (Controlled-NOT) gate for the specified qubit positions.

**Args**
- `ibit` (int): The control qubit position.
- `jbit` (int): The target qubit position.
`n` (int): Total number of qubits in the system.

**Returns**
- `scipy.sparse.csr_matrix`: The CNOT gate as a sparse matrix.

In [19]:
from utilities.quantum_gates import CNOT

ibit = 0
jbit = 1
n = 2
cnot_result = CNOT(ibit, jbit, n)
print(cnot_result.todense())

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


## sampling

### `sample_from_prob`

Samples `num_sample` elements from the dataset `x` based on the given probability distribution `pl`. The probabilities are normalized to ensure they sum to 1 before sampling. It returns the sampled elements from `x`.

**Args**
- `x` (numpy.ndarray): Dataset `x` from which to sample, shape `(n_samples, n_features)`.
- `pl` (numpy.ndarray): Probability distribution over the dataset `x`, shape `(n_samples,)`.
- `num_sample` (int): The number of samples to draw.

**Returns**
- `numpy.ndarray`: The sampled elements from `x`, shape `(num_sample, n_features)`.

In [20]:
from utilities.sampling import sample_from_prob

x = np.array([[1], [2], [3], [4], [5]])
pl = np.array([0.1, 0.2, 0.3, 0.2, 0.2])
num_sample = 3
samples = sample_from_prob(x, pl, num_sample)
print(f"Sampled data points (Test Case 1): {samples}")

Sampled data points (Test Case 1): [[4]
 [2]
 [4]]


### `prob_from_sample`

Computes the empirical probability distribution from a dataset. It counts the occurrences of each element in the dataset and normalizes the counts to produce a probability distribution.

**Args**
- `dataset` (numpy.ndarray): The dataset to compute the probability distribution from.
- `hndim` (int): The number of possible distinct outcomes in the dataset.

**Returns**
- `numpy.ndarray`: The empirical probability distribution, shape `(hndim,)`.

In [21]:
from utilities.sampling import prob_from_sample

dataset = np.array([0, 1, 2, 1, 3, 0, 0, 1, 3, 2])
hndim = 4
empirical_prob = prob_from_sample(dataset, hndim)
print(f"Empirical probability distribution (Test Case 3): {empirical_prob}")

Empirical probability distribution (Test Case 3): [0.3 0.3 0.2 0.2]
